In [ ]:
pip install pyqsp classiq

In [ ]:
Tfrom typing import Tuple

import numpy as np
from pyqsp.angle_sequence import QuantumSignalProcessingPhases

from classiq import *
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences
from classiq.interface.generator import QuantumProgram
from classiq.model.qnum import QNum
from classiq.model.qbit import QBit, QArray
from classiq.model.float import CReal, CArray

# Define number of qubits for fixed-point representation
NUM_QUBITS = 3

# ====================
# Quantum subroutines
# ====================

@qfunc
def projector_cnot(reg: QNum, aux: QBit) -> None:
    aux ^= reg == 0


@qfunc
def u_sqrt(a: QNum, ref: QNum, res: QBit) -> None:
    hadamard_transform(ref)
    res ^= a <= ref


@qfunc
def qsvt_sqrt_polynomial(
    qsvt_phases: CArray[CReal], state: QNum, ref: QNum, ind: QBit, qsvt_aux: QBit
) -> None:
    full_reg = QArray[QBit]()

    within_apply(
        lambda: bind([ind, ref, state], full_reg),
        lambda: qsvt(
            qsvt_phases,
            lambda reg_, aux_: projector_cnot(
                reg_[0 : NUM_QUBITS + 1], aux_
            ),  # indicator on [ind, ref]
            lambda reg_, aux_: projector_cnot(reg_[0], aux_),  # indicator on [ind]
            lambda reg_: u_sqrt(
                reg_[1 + NUM_QUBITS : reg_.len], reg_[1 : 1 + NUM_QUBITS], reg_[0]
            ),
            full_reg,
            qsvt_aux,
        ),
    )


# ================================================
# Phase generation for sqrt(x) on domain [0.1, 1]
# ================================================

sqrt_function = lambda x: np.sqrt(x)
qsp_phases_obj = QuantumSignalProcessingPhases.from_function(
    sqrt_function,
    domain=(0.1, 1.0),
    epsilon=1e-2,
    polynomial_degree=15,
)

phases_list = qsp_phases_obj.phases
print("Generated QSP phases:")
print(phases_list)

# ================================================
# Define full Quantum Program
# ================================================

@qfunc
def main() -> None:
    # Declare inputs
    state = qnum(NUM_QUBITS)  # Value to compute sqrt of
    ref = qnum(NUM_QUBITS)    # Reference for comparison
    ind = qbit()              # Indicator
    aux = qbit()              # QSVT auxiliary qubit
    phase_input = const_array([float(p) for p in phases_list], creal)

    # Call QSVT subroutine
    qsvt_sqrt_polynomial(phase_input, state, ref, ind, aux)


# ================================================
# Compile and Simulate
# ================================================

program = QuantumProgram(main=main)

# Define backend preferences
preferences = ClassiqBackendPreferences(target="qiskit.simulator")

designer = ModelDesigner(backend_preferences=preferences)
execution = ExecutionPreferences(simulate=True)

# Compile the program
result = designer.compile(program=program, execution_preferences=execution)

# Simulate or view the circuit
print("\nCompiled QSVT sqrt circuit:")
result.display()


AttributeError: partially initialized module 'classiq' has no attribute 'interface' (most likely due to a circular import)